In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Dense, Dropout, Input
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, Sequential

In [2]:
train_df = pd.read_csv('train_standard.csv')

In [3]:
train_df = train_df.drop('Unnamed: 0', axis=1)

In [4]:
train_df.head()

,id,cc_no,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cc_txn_amt
0,1,1,2,9.0,5,20000,1.000000,1750.000000,0.000000
1,1,98397,2,9.0,5,20000,1.000000,1750.000000,5000.000000
2,2,2,2,3.0,4,106000,4.346154,9111.538462,1034.615385
3,2,9740,2,3.0,4,106000,4.487805,9514.634146,797.560976
4,3,3,2,1.0,3,29000,3.333333,6652.380952,0.000000


In [5]:
train_id_df = pd.read_csv('train.csv')
train_id_df.head()

,id,income
0,1,20000
1,2,106000
2,3,29000
3,4,61000
4,5,18000


In [6]:
train_df = train_df.drop('cc_no', axis=1)
temp_group = train_df.groupby(['id', 'gender', 'ocp_cd', 'age', 'income']).mean()
temp_group

,,,,,kp_txn_count,kp_txn_amt,cc_txn_amt
id,gender,ocp_cd,age,income,,,
1,2,9.0,5,20000,1.000000,1750.000000,2500.000000
2,2,3.0,4,106000,4.416979,9313.086304,916.088180
3,2,1.0,3,29000,3.333333,6652.380952,0.000000
4,2,3.0,5,61000,12.191071,247372.142857,8785.714286
6,1,3.0,3,19000,3.333333,4466.666667,3833.333333
...,...,...,...,...,...,...,...
49995,1,4.0,6,132000,1.625000,5718.750000,793.750000
49996,2,3.0,4,48000,3.125000,7281.250000,246.875000
49997,1,9.0,5,37000,9.687692,27355.461538,59.615385


In [7]:
df = temp_group.reset_index()
df.head()

,id,gender,ocp_cd,age,income,kp_txn_count,kp_txn_amt,cc_txn_amt
0,1,2,9.0,5,20000,1.000000,1750.000000,2500.000000
1,2,2,3.0,4,106000,4.416979,9313.086304,916.088180
2,3,2,1.0,3,29000,3.333333,6652.380952,0.000000
3,4,2,3.0,5,61000,12.191071,247372.142857,8785.714286
4,6,1,3.0,3,19000,3.333333,4466.666667,3833.333333


In [8]:
len(df)

38046

In [9]:
len(df) * 0.8

30436.800000000003

In [10]:
len(df) * 0.2

7609.200000000001

In [11]:
indices = np.random.randint(len(df), size=len(df))
val = df.iloc[indices[:int(len(df)*0.2)]]
test = df.iloc[indices[int(len(df)*0.2):int(len(df)*0.2)*2]]
train = df.iloc[indices[int(len(df)*0.2)*2:]]

In [12]:
def split_x_y(df):
    X = pd.DataFrame()
    y = pd.DataFrame()
    cols = df.columns
    for col in cols:
        if col == 'income':
            y[col] = df[col]
        elif col == 'id':
            pass
        else:
            X[col] = df[col]
    X = X.to_numpy()
    y = y.to_numpy().reshape(-1)
    return X, y

In [13]:
x_val, y_val = split_x_y(val)
x_test, y_test = split_x_y(test)
x_train, y_train = split_x_y(train)
x_val.shape

(7609, 6)

In [ ]:
class GodLayer(tf.nn.layer):
    

In [17]:

model = Sequential([
    Dense(1024, activation='relu', input_shape=(6, )),
    Dropout(0.7),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='linear')
])

model.summary()
model.compile(optimizer=Adam(learning_rate=1e-10, decay=0.01), loss='mse', metrics=['mse'])

hist = model.fit(x_train, y_train, batch_size=1024, epochs=100,
                validation_data=(x_val, y_val), verbose=1)


Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 1024)              7168      
_________________________________________________________________
dropout_6 (Dropout)          (None, 1024)              0         
_________________________________________________________________
dense_10 (Dense)             (None, 128)               131200    
_________________________________________________________________
dropout_7 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 129       
Total params: 138,497
Trainable params: 138,497
Non-trainable params: 0
_________________________________________________________________
Train on 22828 samples, validate on 7609 samples
Epoch 1/100
22828/22828 [==============================] - 2s 72u

22828/22828 [==============================] - 1s 32us/sample - loss: 5486793884.6413 - mse: 5486794240.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 42/100
22828/22828 [==============================] - 1s 33us/sample - loss: 5476858685.7684 - mse: 5476858880.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 43/100
22828/22828 [==============================] - 1s 34us/sample - loss: 5513703518.3795 - mse: 5513703424.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 44/100
22828/22828 [==============================] - 1s 34us/sample - loss: 5530664166.8351 - mse: 5530663936.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 45/100
22828/22828 [==============================] - 1s 33us/sample - loss: 5475409945.9275 - mse: 5475409920.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 46/100
22828/22828 [==============================] - 1s 34us/sample - loss: 5494489507.3250 - mse: 5494489600.0000 - val_

22828/22828 [==============================] - 1s 33us/sample - loss: 5476359489.4466 - mse: 5476359680.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 88/100
22828/22828 [==============================] - 1s 32us/sample - loss: 5458531569.3317 - mse: 5458531328.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 89/100
22828/22828 [==============================] - 1s 32us/sample - loss: 5408707927.6061 - mse: 5408708096.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 90/100
22828/22828 [==============================] - 1s 32us/sample - loss: 5455952738.4615 - mse: 5455952896.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 91/100
22828/22828 [==============================] - 1s 32us/sample - loss: 5463401287.8164 - mse: 5463401984.0000 - val_loss: 5387419179.5358 - val_mse: 5387419648.0000
Epoch 92/100
22828/22828 [==============================] - 1s 32us/sample - loss: 5665696121.1593 - mse: 5665695744.0000 - val_

In [91]:
pred = model.predict(x_train)

In [101]:
from sklearn import tree
clf = tree.DecisionTreeRegressor()
clf = clf.fit(x_train, y_train)
clf.predict(x_test)

array([48000., 36000., 14000., ..., 19000., 14000., 29000.])

In [117]:
y_test

array([ 48000, 127000,  17000, ...,  19000,  34000,  40000])

In [112]:
from sklearn.ensemble import RandomForestRegressor
regr = RandomForestRegressor(max_depth=10, random_state=0, n_estimators=3000)
regr = clf.fit(x_train, y_train) 

In [116]:
pred = regr.predict(x_test)

In [119]:
eiei = (pred - y_test)
eiei

array([     0., -91000.,  -3000., ...,      0., -23000., -11000.])

In [123]:
eiei = np.array(eiei, dtype=np.int)
eiei

array([     0, -91000,  -3000, ...,      0, -23000, -11000])

In [124]:
eiei = np.where(eiei == 0, True, False)

In [126]:
eiei = list(eiei

AttributeError: 'numpy.ndarray' object has no attribute 'toList'